In [2]:
# Import appropriate modules from the client library.
from googleads import dfp
from collections import defaultdict
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()

sc =SparkContext()
sqlCtx = SQLContext(sc)


#connect to the dfp network
from googleads import dfp
dfp_client = dfp.DfpClient.LoadFromStorage()
network_service = dfp_client.GetService('NetworkService', version='v201708')
current_network = network_service.getCurrentNetwork()


In [2]:

print "Obtaining line items"
# Initialize appropriate service.
client = dfp.DfpClient.LoadFromStorage()
line_item_service = client.GetService('LineItemService', version='v201708')

# Create a statement to select line items.
statement = dfp.StatementBuilder()

data_line = pd.DataFrame(columns=['line_item_id','order_id','line_item_name','start_date_year','start_date_month','start_date_day','end_date_year','end_date_month','end_date_day','budget','currency_code','impressions','clicks','target_country','target_os','target_demo','values','target_audienceid'])
count_line = 0
# Retrieve a small amount of line items at a time, paging
# through until all line items have been retrieved.
while True:
    response_line = line_item_service.getLineItemsByStatement(statement.ToStatement())
    if 'results' in response_line:
        for line_item in response_line['results']:
                targetting = line_item.targeting
                if "stats" in line_item:
                    impression =line_item.stats.impressionsDelivered
                    clicks=line_item.stats.clicksDelivered
                else:
                    impression =None
                    clicks=None
                #get the start dates    
                if "startDateTime" in line_item:
                    start_date_year = line_item.startDateTime.date.year
                    start_date_month = line_item.startDateTime.date.month
                    start_date_day = line_item.startDateTime.date.day
                else:
                    start_date_year = None
                    start_date_month = None
                    start_date_day = None

                    
                 #get the end dates
                if "endDateTime" in line_item:  
                    end_date_year = line_item.endDateTime.date.year
                    end_date_month = line_item.endDateTime.date.month
                    end_date_day = line_item.endDateTime.date.day
                else:
                    end_date_year = None
                    end_date_month = None
                    end_date_day = None
                    
                if "budget" in line_item:
                    budget= line_item.budget.microAmount
                    currency_code = line_item.budget.currencyCode
                else:
                    budget=None
                    currency_code=None
                  
                #geography of target audience
                if 'geoTargeting' in targetting:
                    geo_target =  targetting.geoTargeting
                    if "targetedLocations" in geo_target:
                        for location in geo_target["targetedLocations"]:
                            target_country = location.displayName
                else:
                    target_country = None

                #Technology of target audience
                if 'technologyTargeting' in targetting:
                    tech_target =  targetting.technologyTargeting
                    if  "operatingSystemTargeting" in tech_target:
                        sub_tech_target = tech_target.operatingSystemTargeting 
                        if "operatingSystems" in sub_tech_target :
                            for sub in sub_tech_target["operatingSystems"]:
                                target_os = sub.name
                else:
                    target_os = None

                #customTargeting of target audience
                if 'customTargeting' in targetting:
                    cus_target =  targetting.customTargeting
                    if "children" in cus_target:
                        for sub in cus_target["children"]:
                            if "children" in sub:
                                for fur_sub in sub["children"]:
                                    #audience list
                                    aud_list = []
                                    if "audienceSegmentIds" in fur_sub:
                                        for audience in fur_sub["audienceSegmentIds"]:
                                            aud_list.append(audience)
                                    #custom criteria list 
                                    target_val = []
                                    if "valueIds" in fur_sub:
                                        target_demo = fur_sub.keyId
                                        
                                        for val in fur_sub["valueIds"]:
                                            target_val.append(val)
                                    data_line.loc[count_line] = [str(line_item.id),str(line_item.orderId),str(line_item.name),str(start_date_year),str(start_date_month),str(start_date_day),str(end_date_year),str(end_date_month),str(end_date_day),             
str(budget),str(currency_code),str(impression),str(clicks),str(target_country),str(target_os),str(target_demo),str(target_val),str(aud_list)]
                                    count_line += 1
                else:
                    target_val = None
                    target_demo = None
                    aud_list = None 
                    data_line.loc[count_line] = [str(line_item.id),str(line_item.orderId),str(line_item.name),str(start_date_year),str(start_date_month),str(start_date_day),str(end_date_year),str(end_date_month),str(end_date_day),             
str(budget),str(currency_code),str(impression),str(clicks),str(target_country),str(target_os),str(target_demo),str(target_val),str(aud_list)]
                    count_line += 1
                    
                    
                    
                
                
                print count_line
        statement.offset += statement.limit                
    else:
        break

data_line_df = sqlCtx.createDataFrame(data_line)
data_line_df.write.mode('overwrite').parquet("gs://ds-url-catag/dfp_new/line_items")
print ("line items obtained")


In [11]:
from googleads import dfp


def main(client):
    custom_targeting_service = client.GetService(
      'CustomTargetingService', version='v201711')

    # Create statement to get all targeting keys.
    data_custom = pd.DataFrame(columns=['id','name','displayname','customtargetid'])
    targeting_key_statement = dfp.StatementBuilder()
    
    all_keys = []
    count_two = 0
    data_custom_high = pd.DataFrame(columns=['id','displayname'])
    
    count_value = 0

    # Get custom targeting keys by statement.
    while True:
        response = custom_targeting_service.getCustomTargetingKeysByStatement(
        targeting_key_statement.ToStatement())
        if 'results' in response:
            all_keys.extend(response['results'])
            targeting_key_statement.offset += targeting_key_statement.limit
            for item in response["results"]:
                if item.displayName is not None :
                    data_custom_high.loc[count_two] = [str(item.id),item.displayName.encode('utf-8')] 
                    count_two += 1
                else:
                    data_custom_high.loc[count_two] = [str(item.id),str(item.displayName)] 
                    count_two += 1
                targeting_key_statement.offset += targeting_key_statement.limit
        else:
            break
        

    if all_keys:
        # Create a statement to select custom targeting values.
        query = ('WHERE customTargetingKeyId IN (%s)' %
                 ', '.join([str(key['id']) for key in all_keys]))
        statement = dfp.FilterStatement(query)

        # Retrieve a small amount of custom targeting values at a time, paging
        # through until all custom targeting values have been retrieved.
        while True:
            response = custom_targeting_service.getCustomTargetingValuesByStatement(
              statement.ToStatement())
            if 'results' in response:
                for custom_targeting_value in response['results']:
                  # Print out some information for each custom targeting value.
                    if custom_targeting_value.name and custom_targeting_value.displayName is not None :
                        data_custom.loc[count_value] = [str(custom_targeting_value.id),custom_targeting_value.name.encode('utf-8'),custom_targeting_value.displayName.encode('utf-8'), str(custom_targeting_value.customTargetingKeyId)] 
                        count_value += 1  
                    else:
                        data_custom.loc[count_value] = [str(custom_targeting_value.id),custom_targeting_value.name.encode('utf-8'),str(custom_targeting_value.displayName), str(custom_targeting_value.customTargetingKeyId)] 
                        count_value += 1  
    
                statement.offset += dfp.SUGGESTED_PAGE_LIMIT
            else:
                break
    data_custom_df = sqlCtx.createDataFrame(data_custom)
    data_custom_df.write.mode('overwrite').parquet("gs://ds-url-catag/dfp_new/custom_fields_line/")
    data_custom_high_df = sqlCtx.createDataFrame(data_custom_high)
    data_custom_high_df.write.mode('overwrite').parquet("gs://ds-url-catag/dfp_new/custom_fields_line_high/")


if __name__ == '__main__':
  # Initialize client object.
    dfp_client = dfp.DfpClient.LoadFromStorage()
    main(dfp_client)


Number of results found: 359
